In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

# from simulator import Simulation
from tqdm.auto import tqdm
import itertools
import matplotlib as mpl
from collections import OrderedDict, defaultdict

from voting_mechanism_design.agents.pairwise_badgeholder import PairwiseBadgeholder, PairwiseBadgeholderPopulation
from voting_mechanism_design.projects.project import Project, ProjectPopulation
from voting_mechanism_design.funds_distribution.pairwise_binary import PairwiseBinary
from voting_mechanism_design.sim import RoundSimulation

from voting_mechanism_design.agents.quorum_badgeholder import QuorumBadgeholder, QuorumBadgeholderPopulation
from voting_mechanism_design.funds_distribution.threshold_and_aggregate import ThresholdAndAggregate

# # OP simulator configuration
# from voting_mechanism_design.legacy.op_simulator import Simulation as OPSimulation
# from voting_mechanism_design.legacy.op_simulator import Round as OPRound
# from voting_mechanism_design.legacy.op_simulator import Voter as OPVoter
# from voting_mechanism_design.legacy.op_simulator import Project as OPProject

import itertools
import numpy as np
import choix
import math

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from tqdm.auto import tqdm
from joblib import Parallel, delayed

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [3]:
def create_monotonic_array(max_val, min_val, length, total_sum):
    x = np.linspace(max_val, min_val, length)
    x /= x.sum()
    x *= total_sum
    
    # if any values exceed the first_value, we need to adjust the values
    mm1 = x[0]
    mm2 = x[-1]
    if mm1 > max_val:
        delta = mm1 - max_val
    else:
        delta = 0
    x = np.linspace(mm1-delta, mm2+delta, length)
    x /= x.sum()
    x *= total_sum

    return x

def run_single_quorum_threshold_simulation(
        n_badgeholders=50,
        badgeholder_expertise_vec=None,
        badgeholder_laziness_vec=None,
        n_projects=100,
        project_impact_vec=None,
        # configuration specific to quorum+threshold voting method
        quorum=5,
        scoring_fn='mean',
        min_vote_amt=1,
        max_vote_amt=16,
        max_funding=100,
        random_seed=1234
    ):
    # create voters
    voters = []
    for ii in range(n_badgeholders):
        assert badgeholder_laziness_vec[ii] >= 0
        v = QuorumBadgeholder(
            badgeholder_id=ii,
            total_funds=max_funding,
            min_vote=min_vote_amt,  
            max_vote=max_vote_amt,
            laziness=badgeholder_laziness_vec[ii],
            expertise=badgeholder_expertise_vec[ii],
            coi_factor=0,
            expertise_model="k2"
        )
        voters.append(v)
    badgeholder_pop=QuorumBadgeholderPopulation()
    badgeholder_pop.add_badgeholders(voters)

    # add projects
    projects = []
    for ii in range(n_projects):
        p = Project(
            project_id=ii,
            true_impact=project_impact_vec[ii],  
            owner_id=None,  
        )
        projects.append(p)
    project_population = ProjectPopulation()
    project_population.add_projects(projects)

    # ##  Voting Simulation
    fund_distribution_model = ThresholdAndAggregate()
    simulation_obj = RoundSimulation(
        badgeholder_population=badgeholder_pop,
        projects=project_population,
        funding_design=fund_distribution_model,
    )
    projects = project_population.get_projects()
    simulation_obj.run(cast_votes_kwargs={'projects': projects})

    # extract the scores
    project_population = simulation_obj.projects
    project_vote_counts = defaultdict(int)
    for project in projects:
        for vote in project.votes:
            # project_vote_counts[project.project_id] += round(vote.amount if vote.amount is not None else 0, 10)
            project_vote_counts[project.project_id] += vote.amount if vote.amount is not None else 0
    project_funds_df = pd.DataFrame({
        'Project Id': list(project_vote_counts.keys()),
        'Total Funds': list(project_vote_counts.values())
    })
    sorted_df = project_funds_df.sort_values(by='Total Funds', ascending=False).reset_index(drop=True)
    project_rankings = sorted_df['Project Id'].tolist()
    return _, sorted_df
    # TODO: we need a dictionary

    # return project_funds_df

    # projectid2score = round.calculate_scores(
    #     scoring_fn, 
    #     quorum
    # )
    # projectid2score = OrderedDict(projectid2score)
    # ranks = np.argsort(np.asarray(list(projectid2score.values())))
    # projectid2rank = {ii:ranks[ii] for ii in range(len(ranks))}
    
    # return projectid2score, projectid2rank

def run_single_pairwise_simulation(
        n_badgeholders=50,
        badgeholder_expertise_vec=None,
        badgeholder_laziness_vec=None,
        n_projects=100,
        project_impact_vec=None,
        random_seed=1234
    ):
    # create badgeholders
    badgeholder_list = []
    for ii in range(n_badgeholders):
        badgeholder_id = ii
        expertise = badgeholder_expertise_vec[ii]
        laziness = badgeholder_laziness_vec[ii]
        
        coi_project_ids = []
        coi_factor = 0.0
        agent = PairwiseBadgeholder(
            badgeholder_id=badgeholder_id,
            voting_style='skewed_towards_impact',
            voting_style_kwargs={'use_impact_delta':False},
            expertise=expertise,
            laziness=laziness,
            coi_project_ix_vec=coi_project_ids,
            coi_factor=coi_factor,
        )
        badgeholder_list.append(agent)
    badgeholders = PairwiseBadgeholderPopulation()
    badgeholders.add_badgeholders(badgeholder_list)

    # create projects
    projects = []
    for ii in range(n_projects):
        project_id = ii
        project_impact = project_impact_vec[ii]
        project = Project(
            project_id=project_id,
            true_impact=project_impact,  
            owner_id=None,  # used for COI modeling
        )
        projects.append(project)
    project_population = ProjectPopulation()
    project_population.add_projects(projects)

    # this is a no-op currently, but can change in the future
    fund_distribution_model = PairwiseBinary()

    # create and run a simulation
    simulation_obj = RoundSimulation(
        badgeholder_population=badgeholders,
        projects=project_population,
        funding_design=fund_distribution_model,
        random_seed=random_seed
    )

    # voters can vote on all projects, but an individual badgeholder can decide to ignore
    # some, based on the badgeholder configuration
    voting_view_ix = list(itertools.combinations(range(project_population.num_projects), 2))  # we need a list since we use this twice
    voting_view = []
    for ix1, ix2 in voting_view_ix:
        voting_view.append((project_population.get_project(ix1), project_population.get_project(ix2)))
    simulation_obj.run(cast_votes_kwargs={'view': voting_view})

    # store the project impact differences
    # this is only for diagnostics/post-hoc analysis
    impact_diff_vec = []
    for ix1, ix2 in voting_view_ix:
        p1 = project_population.get_project(ix1)
        p2 = project_population.get_project(ix2)
        impact_diff_vec.append(p1.true_impact - p2.true_impact)

    # make a matrix of all the pairwise voting possibilities
    all_votes = simulation_obj.badgeholder_population.get_all_votes()
    project_list = simulation_obj.projects.projects
    N = len(project_list)
    project_vote_matrix = np.zeros((N, N))
    project2ix = {p:ix for ix, p in enumerate(project_list)}
    vote_data_list = []

    # put the votes into the format needed by the choix package
    for v in all_votes:
        ix1 = project2ix[v.project1]
        ix2 = project2ix[v.project2]
        project_vote_matrix[ix1, ix2] += v.val1
        project_vote_matrix[ix2, ix1] += v.val2

        if v.val1 > v.val2:
            vote_data_list.append((ix1, ix2))
        elif v.val2 > v.val1:
            vote_data_list.append((ix2, ix1))
        else:
            # TODO: not sure if there is a tie, is that even possible??
            print('In Tie scenario!')
            pass

    # estimate parameters of bradley terry model - TODO: update to MCMC estimation
    try:
        params = choix.ilsr_pairwise(N, vote_data_list)  
    except Exception as e:
        # print(f'Failed to estimate global rankings: {e}, adding some regularization!')
        params = choix.ilsr_pairwise(N, vote_data_list, alpha=0.01)
    # create rank order of project ids
    project_ix_worst_to_best_inferred = np.argsort(params)
    project_ix_best_to_worst_inferred = project_ix_worst_to_best_inferred[::-1]

    # # compare the inferred ranks to the actual ranks
    # actual_project_impacts = np.array([p.true_impact for p in project_list])
    # # rank them
    # project_ix_worst_to_best_actual = np.argsort(actual_project_impacts)
    # project_ix_best_to_worst_actual = project_ix_worst_to_best_actual[::-1]

    # return actual_project_impacts, project_ix_worst_to_best_inferred
    return params, project_ix_worst_to_best_inferred, impact_diff_vec

def run_single_simulation(
        n_badgeholders=50,
        badgeholder_expertise_vec=None,
        badgeholder_laziness_vec=None,
        n_projects=100,
        project_impact_vec=None,
        quorum=5,
        scoring_fn='mean',
        min_vote_amt=1,
        max_vote_amt=16,
        max_funding=100,
        random_seed=1234
    ):
    qt_project_scores, qt_project_ranks = run_single_quorum_threshold_simulation(
        n_badgeholders=n_badgeholders,
        badgeholder_expertise_vec=badgeholder_expertise_vec,
        badgeholder_laziness_vec=badgeholder_laziness_vec,
        n_projects=n_projects,
        project_impact_vec=project_impact_vec,
        # configuration specific to quorum+threshold voting method
        quorum=quorum,
        scoring_fn=scoring_fn,
        min_vote_amt=min_vote_amt,
        max_vote_amt=max_vote_amt,
        max_funding=max_funding,
        random_seed=random_seed
    )
    # raise Exception("Test")

    # NOTE: this is a simple version which excludes
    # COI information.  We can build up the simulation to add this
    
    # NOTE: this is a way to align the laziness between Q+T and Pairwise
    # it is imperfect at best, and we need to do some more thinking to determine
    # how better to align this, but this is a starting point.  The thinking here
    # is that a badgeholder of laziness = 0.1 in Q+T is roughly equivalent to a
    # a badgeholder of laziness = 0.01 in pairwise, a 10x conversion factor. The conversion
    badgeholder_laziness_vec_pw = np.power(badgeholder_laziness_vec, 0.2)

    pairwise_project_scores, pairwise_project_ranks, impact_diff_vec = run_single_pairwise_simulation(
        n_badgeholders=n_badgeholders,
        badgeholder_expertise_vec=badgeholder_expertise_vec,
        badgeholder_laziness_vec=badgeholder_laziness_vec_pw,
        n_projects=n_projects,
        project_impact_vec=project_impact_vec,
        random_seed=random_seed
    )

    # the results to return, but others may contain useful diagnostics as well!
    pairwise_inferred_rankings = pairwise_project_ranks
    # qt_inferred_rankings = np.asarray(list(qt_project_ranks.values()))
    qt_inferred_rankings = []
    for ii in range(n_projects):
        qt_inferred_rankings.append(qt_project_ranks[ii])
    qt_inferred_rankings = np.asarray(qt_inferred_rankings)
    true_project_rankings = np.argsort(project_impact_vec)
    
    # compute rank correlation as an initial measure of how aligned 
    tau_pairwise = stats.kendalltau(true_project_rankings, pairwise_inferred_rankings)
    tau_qt = stats.kendalltau(true_project_rankings, qt_inferred_rankings)

    return {
        'rankings': {
            'pairwise': pairwise_inferred_rankings,
            'qt': qt_inferred_rankings,
            'true': true_project_rankings
        },
        'metrics': {
            # index-0 is the actual statistic, index-1 is a p-value or something
            'pairwise': tau_pairwise[0],
            'qt': tau_qt[0]
        }
    }

def run_n_simulations(
        n_sims,
        n_badgeholders=50,
        badgeholder_expertise_vec=None,
        badgeholder_laziness_vec=None,
        n_projects=100,
        project_impact_vec=None,
        quorum=5,
        scoring_fn='mean',
        min_vote_amt=1,
        max_vote_amt=16,
        max_funding=100,
        random_seed_start=1234,
        verbose=False,
    ):
    def run_simulation(ii):
        seed = random_seed_start + ii
        single_run_results = run_single_simulation(
            n_badgeholders=n_badgeholders,
            badgeholder_expertise_vec=badgeholder_expertise_vec,
            badgeholder_laziness_vec=badgeholder_laziness_vec,
            n_projects=n_projects,
            project_impact_vec=project_impact_vec,
            quorum=quorum,
            scoring_fn=scoring_fn,
            min_vote_amt=min_vote_amt,
            max_vote_amt=max_vote_amt,
            max_funding=max_funding,
            random_seed=seed
        )
        # return a dict so that we can create a dataframe of the results
        # for each configuration we test quickly
        return_dict = {
            'Pairwise': single_run_results['metrics']['pairwise'],
            'Q+T': single_run_results['metrics']['qt'],
            # 'pw_rank': single_run_results['rankings']['pairwise'],
            # 'qt_rank': single_run_results['rankings']['qt'],
            'n_badgeholders': n_badgeholders,
            'avg_expertise': np.mean(badgeholder_expertise_vec),
            'avg_laziness': np.mean(badgeholder_laziness_vec),
            'n_projects': n_projects,
            'quorum': quorum,
            'scoring_fn': scoring_fn,
            'min_vote_amt': min_vote_amt,
            'max_vote_amt': max_vote_amt,
            'max_funding': max_funding,
        }
        return return_dict
    
    results = Parallel(n_jobs=-1)(delayed(run_simulation)(ii) for ii in tqdm(range(n_sims), disable=(not verbose)))
    # results = []
    # for ii in tqdm(range(n_sims), disable=(not verbose)):
    #     results.append(run_simulation(ii))

    return pd.DataFrame(results)


In [4]:
# test w/ some random values to start
n_badgeholders = 25
n_projects = 100
badgeholder_expertise_vec = 0.25*np.ones(n_badgeholders)
badgeholder_laziness_vec = 0.25*np.ones(n_badgeholders)
project_impact_vec=np.linspace(0, 1, n_projects)  # enforce a strict ranking that we can compare against

quorum = 5
scoring_fn = 'mean'
min_vote_amt = 1
max_vote_amt = 16
max_funding = 100
random_seed = 1234

qt_project_scores, qt_project_ranks = run_single_quorum_threshold_simulation(
        n_badgeholders=n_badgeholders,
        badgeholder_expertise_vec=badgeholder_expertise_vec,
        badgeholder_laziness_vec=badgeholder_laziness_vec,
        n_projects=n_projects,
        project_impact_vec=project_impact_vec,
        # configuration specific to quorum+threshold voting method
        quorum=quorum,
        scoring_fn=scoring_fn,
        min_vote_amt=min_vote_amt,
        max_vote_amt=max_vote_amt,
        max_funding=max_funding,
        random_seed=random_seed
    )

In [6]:
qt_project_ranks

,Project Id,Total Funds
0,1,37.5
1,17,37.5
2,10,37.5
3,4,37.5
4,34,37.5
...,...,...
95,75,0.0
96,83,0.0
97,76,0.0
98,71,0.0


In [ ]:
# setup simulation parameters
n_sims = 32

n_badgeholders_sweep = [25]
n_projects_sweep = [100]
badgeholder_expertise_sweep = [0.0, 0.25, 0.5, 0.75, 1.0]
badgeholder_laziness_sweep = [0.0, 0.25, 0.5, 0.75]

experiment_cfgs = list(itertools.product(
    n_badgeholders_sweep,
    n_projects_sweep,
    badgeholder_expertise_sweep,
    badgeholder_laziness_sweep
))

# parameters that we can explore
quorum=5
scoring_fn='mean'
min_vote_amt=1
max_vote_amt=16
max_funding=100

all_results = []
for experiment_cfg in tqdm(experiment_cfgs):
    n_badgeholders=experiment_cfg[0]
    n_projects=experiment_cfg[1]

    badgeholder_expertise_vec=np.ones(n_badgeholders) * experiment_cfg[2]
    badgeholder_laziness_vec=np.ones(n_badgeholders) * experiment_cfg[3]

    random_seed_start=1234
    project_impact_vec=np.linspace(0, 1, n_projects)  # enforce a strict ranking that we can compare against

    results_df = run_n_simulations(
        n_sims,
        n_badgeholders=n_badgeholders,
        badgeholder_expertise_vec=badgeholder_expertise_vec,
        badgeholder_laziness_vec=badgeholder_laziness_vec,
        n_projects=n_projects,
        project_impact_vec=project_impact_vec,
        quorum=quorum,
        scoring_fn=scoring_fn,
        min_vote_amt=min_vote_amt,
        max_vote_amt=max_vote_amt,
        max_funding=max_funding,
        random_seed_start=random_seed_start,
        verbose=False
    )
    all_results.append(results_df)
all_results = pd.concat(all_results)

In [5]:
dff = pd.melt(
    all_results, 
    value_vars=['Pairwise', 'Q+T'], value_name='Rank Corr', var_name='Mechanism',
    id_vars=['n_badgeholders', 'avg_expertise', 'avg_laziness', 'n_projects', 'quorum']
)
dff['Hue'] = ''
for ii, row in dff.iterrows():
    out_str = '%s//%0.02f' % (row['Mechanism'], row['avg_laziness'],)
    dff.at[ii, 'Hue'] = out_str

In [ ]:
dff

In [ ]:
# create palette
mechanisms = ['Pairwise', 'Q+T']
laziness_vec = [0.0, 0.25, 0.5, 0.75]
color_range = np.linspace(0.8, 0.2, len(laziness_vec))
colors=['Blues', 'Greens', 'Reds', 'Purples']
palette = {}
for jj, m in enumerate(mechanisms):
    color = mpl.colormaps[colors[jj]]
    for ii, l in enumerate(laziness_vec):
        key='%s//%0.02f' % (m, l,)
        palette[key] = color(color_range[ii])

ax = sns.stripplot(
    data=dff, x="avg_expertise", y="Rank Corr", hue="Hue",
    dodge=True, zorder=1, legend=False, palette=palette,
)
plt.xlabel('Expertise')
plt.ylabel('Alignment')
# plt.legend(title='Mechanism//Laziness')
# create custom legend
for ii, m in enumerate(mechanisms):
    ax.scatter([], [], color=mpl.colormaps[colors[ii]](0.8), label=mechanisms[ii])
for ii, l in enumerate(laziness_vec):
    ax.scatter([], [], color=mpl.colormaps['Greys'](color_range[ii]), label='Lz=%0.02f' % l)
ax.legend(loc='lower right')

#### Notes on Alignment for Q+T
1. Q+T has less alignment than Pairwise.  Even if the expertise is normalized, it is less b/c
   the user must also assign funding amounts to each project.  This introduces an additional source
   of randomness.
2. Even if it is not random (i.e. top ranked project gets top amount and then work down), all badgeholder's views are aggregated and this introduces some noise in the alignment.
3. Interestingly, as badgeholder laziness goes up in the Q+T scheme, the alignment seems to get better. Likely b/c they vote on less projects, so there is less ambiguity in assigning projects closer amounts of funding, causing
4. It really depends on how a Q+T badgeholder assigns funding to the projects after they have ranked them.
5. This means the above plot may not be completely fair in terms of comparison, in the sense that pairwise is comparing alignment directly, where as Q+T is including the scoring aspect.  That being said, that is sort of the mechanism, since in pairwise, we only want rankings and then we create an a-priori mapping to funding amount, so this is sort of an advantage in the sense that it creates a clearer signal.
